#### 电影推荐：
##### 使用亲和性分析方法  基于apriori算法分析

In [1]:
import os
#使用pandas加载数据
import pandas as pd

data_folder=os.path.join(os.path.expanduser("~"),'ml-100k')
ratings_filename=os.path.join('u.data')
all_ratings=pd.read_csv(ratings_filename,delimiter='\t',header=None,names=['UserID','MovieID','Rating','Datetime'])
print(all_ratings.info())
all_ratings[:1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
UserID      100000 non-null int64
MovieID     100000 non-null int64
Rating      100000 non-null int64
Datetime    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB
None


,UserID,MovieID,Rating,Datetime
0,196,242,3,881250949


In [2]:
#转换时间格式
all_ratings['Datetime']=pd.to_datetime(all_ratings['Datetime'],unit='s')
all_ratings[:1]

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49


In [3]:
#新增一列，用来存用户对某个电影是否喜欢  （如何评分大于3则是）
all_ratings['Favorable']=all_ratings['Rating']>3
all_ratings.head()
all_ratings.describe()

,UserID,MovieID,Rating
count,100000.00000,100000.000000,100000.000000
mean,462.48475,425.530130,3.529860
std,266.61442,330.798356,1.125674
min,1.00000,1.000000,1.000000
25%,254.00000,175.000000,3.000000
50%,447.00000,322.000000,4.000000
75%,682.00000,631.000000,4.000000
max,943.00000,1682.000000,5.000000


In [4]:
#从数据中取出userid为前200名用户的打分数据作训练集
ratings=all_ratings[all_ratings['UserID'].isin(range(201) ) ]
#过滤一次数据，只保留用户喜欢的电影即可
favorable_ratings=ratings[ratings["Favorable"]]
favorable_ratings.head()

,UserID,MovieID,Rating,Datetime,Favorable
12,200,222,5,1997-10-05 09:05:40,True
16,122,387,5,1997-11-11 17:47:39,True
20,119,392,4,1998-01-30 16:13:34,True
21,167,486,4,1998-04-16 14:54:12,True
26,38,95,5,1998-04-13 01:14:54,True


In [5]:
#生成频繁项集，所以我们只对打分超过一次的用户感兴趣，所以按照UserId分组，并且遍历每个用户看过的每一部剧
#格式  ： 用户 id  ：{电影集合}
from collections import defaultdict
favorable_reviews_by_users=dict((k,frozenset(v.values))
                               for k,v in favorable_ratings.groupby('UserID')['MovieID'])
print(favorable_reviews_by_users)
print('有{0}个用户的数据'.format(len(favorable_reviews_by_users)))

{1: frozenset({1, 3, 6, 7, 9, 12, 13, 14, 15, 16, 18, 19, 20, 22, 23, 25, 28, 32, 33, 39, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 55, 56, 57, 58, 59, 60, 61, 64, 65, 66, 68, 72, 75, 76, 77, 79, 80, 81, 82, 84, 86, 87, 88, 89, 90, 91, 93, 95, 96, 98, 100, 106, 107, 108, 109, 111, 113, 114, 115, 119, 121, 123, 124, 127, 128, 129, 132, 133, 134, 135, 137, 144, 146, 150, 151, 152, 154, 156, 157, 160, 161, 162, 163, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 181, 182, 183, 184, 185, 186, 187, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 202, 203, 204, 206, 207, 208, 209, 210, 212, 214, 216, 221, 222, 223, 224, 227, 228, 229, 230, 234, 235, 236, 238, 239, 241, 242, 246, 248, 249, 250, 251, 253, 256, 257, 258, 265, 267, 268, 269, 270}), 2: frozenset({257, 1, 13, 14, 269, 272, 273, 275, 276, 277, 279, 25, 282, 283, 284, 285, 286, 292, 293, 295, 297, 299, 300, 301, 302, 303, 304, 50, 306, 310, 311, 313, 316, 100, 237, 111, 242, 251, 127, 255}), 3: frozenset({320, 321,

In [6]:
#在创建一个数据框  存入每部电影评分为3以上的人数  即 favorable 为TRUe的数量
num_favorable_by_movie=ratings[['MovieID','Favorable']].groupby('MovieID').sum()
#看结果
num_favorable_by_movie.sort_values('Favorable',ascending=False)[:5]

,Favorable
MovieID,
50,101.0
100,89.0
258,84.0
181,79.0
174,75.0


In [7]:
#创建一个函数，他接收新发现的频繁项集，创建超集  检测频繁程度
'''
favorable_reviews_by_users:用户打分情况的集合  {用户名： frozenset(电影名，电影号)}
k_1_itemsets:上一个频繁项集
minsupport:最小支持度
返回值格式：
    dict(频繁项集  支持度)
'''
def find_frequent_itemsets(favorable_reviews_by_users,k_1_itemsets,min_support):
    counts=defaultdict(int)
    #循环用户和他们的打分数据
    for user,reviews in favorable_reviews_by_users.items():
        #循环前一次找出频繁项集，判断他们是否为reviews的子集
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):  #如果是，表名用户已经为子集中的电影打过分了
                for other_reviewed_movie in reviews-itemset:  #other_reviewed_movie  用户打过分，但不在频繁项集里面
                    current_superset=itemset|frozenset((other_reviewed_movie,))
                    counts[current_superset]+=1   #这个频繁项集的数量支持度+1
    #函数最后检测达到支持度要求的项集，返回频繁度够的频繁项集
    return dict([(itemset,frequency) for itemset,frequency in counts.items() if frequency>=min_support ] )

In [15]:
#测试   用已知的 k_1 项集 生成 k_2 项集
#这里的候选项的产生使用了  fk-1* f1
k_2_frequence_list=find_frequent_itemsets(favorable_reviews_by_users,frequent_itemsets[1],min_support)
print(k_2_frequence_list)

{frozenset({1, 7}): 64, frozenset({1, 50}): 102, frozenset({56, 1}): 66, frozenset({64, 1}): 60, frozenset({1, 79}): 64, frozenset({1, 98}): 74, frozenset({1, 100}): 84, frozenset({1, 121}): 58, frozenset({1, 127}): 66, frozenset({1, 172}): 62, frozenset({1, 174}): 84, frozenset({1, 181}): 76, frozenset({1, 222}): 60, frozenset({1, 258}): 50, frozenset({9, 7}): 52, frozenset({50, 7}): 96, frozenset({56, 7}): 80, frozenset({64, 7}): 64, frozenset({79, 7}): 70, frozenset({98, 7}): 76, frozenset({100, 7}): 94, frozenset({121, 7}): 60, frozenset({127, 7}): 58, frozenset({172, 7}): 72, frozenset({174, 7}): 80, frozenset({181, 7}): 72, frozenset({222, 7}): 62, frozenset({258, 7}): 70, frozenset({9, 50}): 72, frozenset({56, 9}): 62, frozenset({64, 9}): 58, frozenset({9, 98}): 60, frozenset({9, 100}): 78, frozenset({9, 127}): 66, frozenset({9, 174}): 56, frozenset({9, 181}): 52, frozenset({56, 50}): 94, frozenset({64, 50}): 82, frozenset({50, 79}): 86, frozenset({50, 98}): 98, frozenset({50, 1

In [10]:
import sys
#创建一个字典  存不同长度的频繁项集
#数据格式：
#         频繁项集长度   对应的频繁项集
frequent_itemsets={}
min_support=50  #要求最小的支持度
#从频繁项集长度为1，支持度要大于50
frequent_itemsets[1]=dict((frozenset((movie_id,)),row['Favorable'])for movie_id,row in num_favorable_by_movie.iterrows()
                         if row["Favorable"]>min_support)
#输出频繁项集长度为1  支持度大于50的所有电影信息
frequent_itemsets[1]
print('there are {0} movies  with more than {1} favorable reviews'.format(len(frequent_itemsets[1]),min_support))
print(frequent_itemsets[1].items())
sys.stdout.flush()

there are 19 movies  with more than 50 favorable reviews
dict_items([(frozenset({1}), 67.0), (frozenset({7}), 68.0), (frozenset({9}), 54.0), (frozenset({50}), 101.0), (frozenset({56}), 68.0), (frozenset({64}), 58.0), (frozenset({79}), 59.0), (frozenset({98}), 71.0), (frozenset({100}), 89.0), (frozenset({121}), 51.0), (frozenset({127}), 70.0), (frozenset({172}), 60.0), (frozenset({174}), 75.0), (frozenset({181}), 79.0), (frozenset({222}), 51.0), (frozenset({258}), 84.0), (frozenset({286}), 60.0), (frozenset({313}), 61.0), (frozenset({318}), 51.0)])


In [11]:
#定义要找的频繁集的最大长度
max_length=20
#循环集长度从2到max_length
for k in range(2,max_length):
    cur_frequent_itemsets=find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],min_support )
    if len(cur_frequent_itemsets)==0:
        print("can not find any frequent itemsets of length{0}".format(k))
        sys.stdout.flush()
        break
    else:
        print('find{0} frequent itemsets of length {1}'.format(len(cur_frequent_itemsets),k))
        print('\t data as following:')
        sys.stdout.flush()
        frequent_itemsets[k]=cur_frequent_itemsets

find129 frequent itemsets of length 2
	 data as following:
find441 frequent itemsets of length 3
	 data as following:
find916 frequent itemsets of length 4
	 data as following:
find1209 frequent itemsets of length 5
	 data as following:
find1015 frequent itemsets of length 6
	 data as following:
find534 frequent itemsets of length 7
	 data as following:
find169 frequent itemsets of length 8
	 data as following:
find29 frequent itemsets of length 9
	 data as following:
find2 frequent itemsets of length 10
	 data as following:
can not find any frequent itemsets of length11


In [12]:
'''
以上的apriori算法结束后  得到一系列的频繁项集，但还不是关联规则，频繁项集是一组达到最小支持度的项目，而关联规则是由前提与结论组成的
从频繁项集中抽取相关联的规则，把其中几部电影作为前提，另外几部作为结论规则：如果用户喜欢xx，yy，zz那么他们也会喜欢ttt
'''
candidate_rules=[]   #关联规则结构    [(frozenset(前提),结论)，(frozenset(前提),结论)]
#itemset_length  频繁项集长度
#itemset_counts  
for itemset_length,itemset_counts in frequent_itemsets.items():
    #取出itemset_counts中的每个键，[电影1，电影2，.....]
    for itemset in itemset_counts.keys():
        #循环频繁项集中的每部电影，生成条件和结论
        for conclusion in itemset:
            premise=itemset-set((conclusion,))
            candidate_rules.append((premise,conclusion))
print("总共有{0}条备选规则".format(len(candidate_rules)))
print("规则细节如下：")
print(candidate_rules)

总共有22770条备选规则
规则细节如下：
[(frozenset(), 1), (frozenset(), 7), (frozenset(), 9), (frozenset(), 50), (frozenset(), 56), (frozenset(), 64), (frozenset(), 79), (frozenset(), 98), (frozenset(), 100), (frozenset(), 121), (frozenset(), 127), (frozenset(), 172), (frozenset(), 174), (frozenset(), 181), (frozenset(), 222), (frozenset(), 258), (frozenset(), 286), (frozenset(), 313), (frozenset(), 318), (frozenset({7}), 1), (frozenset({1}), 7), (frozenset({50}), 1), (frozenset({1}), 50), (frozenset({1}), 56), (frozenset({56}), 1), (frozenset({1}), 64), (frozenset({64}), 1), (frozenset({79}), 1), (frozenset({1}), 79), (frozenset({98}), 1), (frozenset({1}), 98), (frozenset({100}), 1), (frozenset({1}), 100), (frozenset({121}), 1), (frozenset({1}), 121), (frozenset({127}), 1), (frozenset({1}), 127), (frozenset({172}), 1), (frozenset({1}), 172), (frozenset({174}), 1), (frozenset({1}), 174), (frozenset({181}), 1), (frozenset({1}), 181), (frozenset({222}), 1), (frozenset({1}), 222), (frozenset({258}), 1), (

In [13]:
'''
计算每条规则的置信度   步骤1  先计算正确的规则数（支持度）   错误规则数
'''
correct_counts=defaultdict(int)   #规则应验
incorrect_counts=defaultdict(int)  #规则不适用
#循环所有的用户及他们喜欢的电影
for user,reviews in favorable_reviews_by_users.items():
    #循环所有的规则
    for candidate_rule in candidate_rules:
        premise,conclusion=candidate_rule
        #判断前提是否是review中的一个子类，并且结论也在 review 中 说明这条规则应验  否则不适用
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule]+=1
            else:
                incorrect_counts[candidate_rule]+=1
                
#计算置信度
rule_confidence={candidate_rule:
                 correct_counts[candidate_rule]/float(correct_counts[candidate_rule]+incorrect_counts[candidate_rule])
                 for candidate_rule in candidate_rules}

#设置最低置信度
min_confidence=0.95
#过滤掉小于最低置信度的规则
rule_confidence={candidate_rule:  confidence for candidate_rule,confidence in rule_confidence.items()
                if confidence>min_confidence}
print('大于最低置信度{0}的有效规则总数有{1}'.format(min_confidence,len(rule_confidence ) ) )

大于最低置信度0.95的有效规则总数有3747


In [14]:
#排序输出前N条置信度最高的规则
n=50
from operator import itemgetter
sorted_confidence=sorted(rule_confidence.items(),key=itemgetter(1),reverse=True)
for index in range(n):
    print('第#{0}条规则'.format(index+1) )
    (premise,conclusion)=sorted_confidence[index][0]
    print('规则：前提 {0} 结果{1}'.format(premise,conclusion) )
    print('--置信度为:{0:.3f}'.format(rule_confidence[(premise,conclusion)]))
    print(" ")

第#1条规则
规则：前提 frozenset({98, 181}) 结果50
--置信度为:1.000
 
第#2条规则
规则：前提 frozenset({172, 222}) 结果50
--置信度为:1.000
 
第#3条规则
规则：前提 frozenset({172, 79}) 结果174
--置信度为:1.000
 
第#4条规则
规则：前提 frozenset({172, 222}) 结果174
--置信度为:1.000
 
第#5条规则
规则：前提 frozenset({258, 172}) 结果174
--置信度为:1.000
 
第#6条规则
规则：前提 frozenset({1, 181, 7}) 结果50
--置信度为:1.000
 
第#7条规则
规则：前提 frozenset({1, 222, 7}) 结果50
--置信度为:1.000
 
第#8条规则
规则：前提 frozenset({1, 172, 7}) 结果174
--置信度为:1.000
 
第#9条规则
规则：前提 frozenset({56, 1, 50}) 结果174
--置信度为:1.000
 
第#10条规则
规则：前提 frozenset({56, 1, 181}) 结果50
--置信度为:1.000
 
第#11条规则
规则：前提 frozenset({1, 98, 181}) 结果50
--置信度为:1.000
 
第#12条规则
规则：前提 frozenset({1, 172, 181}) 结果50
--置信度为:1.000
 
第#13条规则
规则：前提 frozenset({1, 172, 222}) 结果50
--置信度为:1.000
 
第#14条规则
规则：前提 frozenset({1, 258, 172}) 结果50
--置信度为:1.000
 
第#15条规则
规则：前提 frozenset({1, 174, 222}) 结果50
--置信度为:1.000
 
第#16条规则
规则：前提 frozenset({1, 258, 174}) 结果50
--置信度为:1.000
 
第#17条规则
规则：前提 frozenset({56, 1, 64}) 结果98
--置信度为:1.000
 
第#18条规则
规则：前提 frozenset({56, 1